# Performance Check

In [1]:
import timeit
import numpy as np
from npgeohash import npgeohash, npgeohash_jit, npgeohash_rs

In [2]:
poi = np.array(
    [
        [35.69240645093, 139.7034750767164],
        [35.691255831981294, 139.69025228754268],
        [35.68307317410146, 139.71622562341963],
        [35.70071598380815, 139.69749333875686],
        [35.68997734701496, 139.6847427920536],
        [35.68115524225217, 139.68584469115146],
    ]
)
lat, lon = 35.68952987243547, 139.69953972279566

results = {}


def _t(stmt):
    print(eval(stmt, globals()))
    t = timeit.Timer(stmt, globals=globals())
    n = 10000
    u = t.timeit(n) / n * 1e6
    print(f"{u:.2f} us")
    results[stmt] = u

The following function is jitted.

In [3]:
_t("npgeohash.encode_array(poi, 7)")
_t("npgeohash_jit.encode_array(poi, 7)")
_t("npgeohash_rs.encode_array(poi, 7)")

['xn77518' 'xn7749r' 'xn76grf' 'xn774gh' 'xn77495' 'xn76fxs']
73.90 us
['xn77518' 'xn7749r' 'xn76grf' 'xn774gh' 'xn77495' 'xn76fxs']
6.13 us
['xn77518', 'xn7749r', 'xn76grf', 'xn774gh', 'xn77495', 'xn76fxs']
1.86 us


In [4]:
_t("list(npgeohash.create_circle(lat, lon, 1000, 7))")
_t("list(npgeohash_jit.create_circle(lat, lon, 1000, 7))")
_t("list(npgeohash_rs.create_circle(lat, lon, 1000, 7))")

['xn7748q', 'xn7748w', 'xn7748y', 'xn7749n', 'xn7749q', 'xn7749w', 'xn7749y', 'xn7748p', 'xn7748r', 'xn7748x', 'xn7748z', 'xn7749p', 'xn7749r', 'xn7749x', 'xn7749z', 'xn774dp', 'xn76fzb', 'xn774b0', 'xn774b2', 'xn774b8', 'xn774bb', 'xn774c0', 'xn774c2', 'xn774c8', 'xn774cb', 'xn774f0', 'xn774f2', 'xn76fz9', 'xn76fzc', 'xn774b1', 'xn774b3', 'xn774b9', 'xn774bc', 'xn774c1', 'xn774c3', 'xn774c9', 'xn774cc', 'xn774f1', 'xn774f3', 'xn76fzd', 'xn76fzf', 'xn774b4', 'xn774b6', 'xn774bd', 'xn774bf', 'xn774c4', 'xn774c6', 'xn774cd', 'xn774cf', 'xn774f4', 'xn774f6', 'xn774fd', 'xn76fze', 'xn76fzg', 'xn774b5', 'xn774b7', 'xn774be', 'xn774bg', 'xn774c5', 'xn774c7', 'xn774ce', 'xn774cg', 'xn774f5', 'xn774f7', 'xn774fe', 'xn76fzk', 'xn76fzs', 'xn76fzu', 'xn774bh', 'xn774bk', 'xn774bs', 'xn774bu', 'xn774ch', 'xn774ck', 'xn774cs', 'xn774cu', 'xn774fh', 'xn774fk', 'xn774fs', 'xn76fzm', 'xn76fzt', 'xn76fzv', 'xn774bj', 'xn774bm', 'xn774bt', 'xn774bv', 'xn774cj', 'xn774cm', 'xn774ct', 'xn774cv', 'xn774fj'

In [5]:
poi_geohashes = npgeohash.encode_array(poi, 7)

_t("npgeohash.neighbors(poi_geohashes[0])")
_t("npgeohash_jit.neighbors(poi_geohashes[0])")
_t("npgeohash_rs.neighbors(poi_geohashes[0])")

['xn77518', 'xn7751b', 'xn774cz', 'xn774cx', 'xn774cr', 'xn77512', 'xn77513', 'xn77519', 'xn7751c']
64.75 us
['xn77518', 'xn7751b', 'xn774cz', 'xn774cx', 'xn774cr', 'xn77512', 'xn77513', 'xn77519', 'xn7751c']
10.46 us
['xn77518', 'xn7751b', 'xn774cz', 'xn774cx', 'xn774cr', 'xn77512', 'xn77513', 'xn77519', 'xn7751c']
1.96 us


The following functions are actually not jitted.

In [6]:
_t("npgeohash.many_neighbors(poi_geohashes)")
_t("npgeohash_jit.many_neighbors(poi_geohashes)")
_t("npgeohash_rs.many_neighbors(poi_geohashes)")

{'xn774fg', 'xn76gre', 'xn77525', 'xn774fv', 'xn76fxg', 'xn76fxv', 'xn77519', 'xn7749h', 'xn76grf', 'xn7749r', 'xn774gh', 'xn7748u', 'xn76fxm', 'xn774gj', 'xn76gr9', 'xn7749p', 'xn77494', 'xn77513', 'xn7749x', 'xn77497', 'xn76fx7', 'xn77524', 'xn774g5', 'xn76fxu', 'xn77518', 'xn77512', 'xn76fxs', 'xn7749k', 'xn774fu', 'xn7748f', 'xn7751b', 'xn774c8', 'xn7748g', 'xn774g7', 'xn77495', 'xn774cx', 'xn76grc', 'xn774cz', 'xn76fxt', 'xn774c0', 'xn76grg', 'xn76fxe', 'xn76grd', 'xn7749n', 'xn76fxk', 'xn774gk', 'xn7749w', 'xn77521', 'xn77496', 'xn7751c', 'xn774gm', 'xn774cr', 'xn774c2', 'xn7749q'}
405.67 us
{'xn774fg', 'xn76gre', 'xn77525', 'xn774fv', 'xn76fxg', 'xn76fxv', 'xn77519', 'xn7749h', 'xn76grf', 'xn7749r', 'xn774gh', 'xn7748u', 'xn76fxm', 'xn774gj', 'xn76gr9', 'xn7749p', 'xn77494', 'xn77513', 'xn7749x', 'xn77497', 'xn76fx7', 'xn77524', 'xn774g5', 'xn76fxu', 'xn77518', 'xn77512', 'xn76fxs', 'xn7749k', 'xn774fu', 'xn7748f', 'xn7751b', 'xn774c8', 'xn7748g', 'xn774g7', 'xn77495', 'xn774cx'

In [7]:
_t("npgeohash.isin(poi_geohashes, poi_geohashes)")
_t("npgeohash_jit.isin(poi_geohashes, poi_geohashes)")
_t("npgeohash_rs.isin(poi_geohashes, poi_geohashes)")

[ True  True  True  True  True  True]
52.64 us
[ True  True  True  True  True  True]
147.06 us
[True, True, True, True, True, True]
40.36 us


In [8]:
_t("npgeohash.isin_circle(poi_geohashes, lat, lon, 1000, 7)")
_t("npgeohash_jit.isin_circle(poi_geohashes, lat, lon, 1000, 7)")
_t("npgeohash_rs.isin_circle(poi_geohashes, lat, lon, 1000, 7)")

[ True  True False False False False]
3402.98 us
[ True  True False False False False]
4622.43 us
[True, True, False, False, False, False]
51.44 us


# Result

In [9]:
from IPython.core.display import Markdown

r_pure = {}
r_jit = {}
r_rust = {}
for func, time in results.items():
    i = func.find("npgeohash")
    j = func.find(".", i)
    k = func.find("(", j)
    module_name = func[i:j]
    func_name = func[j + 1:k]
    if module_name == "npgeohash_jit":
        r_jit[func_name] = time
    elif module_name == "npgeohash_rs":
        r_rust[func_name] = time
    else:
        r_pure[func_name] = time
table = ["Function|npgeohash(wo/ jit)|npgeohash_jit(w/ jit)|Times|npgeohash_rs|Times", "---|---|---|---|---|---"]
for func_name in r_pure.keys():
    table.append(
        "{}|{:.2f}|{:.2f}|{:.2f}|{:.2f}|{:.2f}".format(
            func_name,
            r_pure[func_name],
            r_jit[func_name],
            r_pure[func_name] / r_jit[func_name],
            r_rust[func_name],
            r_pure[func_name] / r_rust[func_name],
        )
    )
display(Markdown("\n".join(table)))
    

Function|npgeohash(wo/ jit)|npgeohash_jit(w/ jit)|Times|npgeohash_rs|Times
---|---|---|---|---|---
encode_array|73.90|6.13|12.06|1.86|39.75
create_circle|1718.44|145.55|11.81|53.24|32.27
neighbors|64.75|10.46|6.19|1.96|33.06
many_neighbors|405.67|377.32|1.08|29.83|13.60
isin|52.64|147.06|0.36|40.36|1.30
isin_circle|3402.98|4622.43|0.74|51.44|66.15